<a href="https://colab.research.google.com/gist/sadov/05fae93ce77a23b0c3cbc6ba91d97f9c/dateno-deep-research-workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Пример Dateno Deep Research workflow

Пример workflow для проведения глубокого исследования с помощью LLM-агентов Dateno. В этом примере можно увидеть как компоновать различные LLM-агенты Dateno для получения управляемого на каждом этапе цикла рабочих исследований с использованием детерменированного поиска в БД датасетов Dateno.

## Первый этап workflow -- Query Assistant LLM-агент

LLM-агент Query Assistant -- анализирует исходный запрос пользователя, определяет ключевые сущности, тему и контекст поиска. Формулирует уточняющие вопросы для устранения неоднозначности и сбора дополнительной информации. Переформулирует запрос в набор запросов для LLM-агента Dateno Search.

In [ ]:
HF_TOKEN='YOUR DATENO HF SPACE TOKEN'

In [ ]:
!pip install git+https://github.com/datenoio/datenollm

  Cloning https://github.com/datenoio/datenollm to /tmp/pip-req-build-qjoyuv1k
  Running command git clone --filter=blob:none --quiet https://github.com/datenoio/datenollm /tmp/pip-req-build-qjoyuv1k
  Resolved https://github.com/datenoio/datenollm to commit c5f990af657cf54ddd03158a426e45bed78cf478
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
  Created wheel for datenollm: filename=datenollm-0.0.1-py3-none-any.whl size=27078 sha256=49075aca4ee59d021f3ed5bcc207f19175a984f6fa92fdab532d71a91ebf8062
  Stored in directory: /tmp/pip-ephem-wheel-cache-fjiksotl/wheels/e0/df/dd/8a6b3b355ce7d5a74ee77bb4f2ae23d95df4edbca4ca18871c
Successfully built datenollm


Проверим есть ли у нас доступ Google Disk'у. Если его нет, то последует запрос на выдачу разрешений для доступа к Google Account'у.

Google запросит очень много разрешений. Однако, если выставить только разрешение на доступ к файлам и каталогам, то подключения к Google Disk не происходит.

По умолчанию для сохранения в файлах истории и контекста мы используем только каталог /content/drive/MyDrive/colab_data/dateno/, ну или каталог указанный в переменной среды окружения `DRIVE_PATH`, ничего больше. См. https://github.com/datenoio/datenollm/blob/main/datenollm/file_utils.py

Если условие предоставления доступа к данным на персональном Google Disk'е является критическим, можно порекомендовать запускать данный Google Collab notebook на другом аккаунте Google, где важных данных нет.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


После предоставления доступа проверяем есть ли каталог /content/drive/MyDrive/colab_data/dateno/, если нет -- он будет создан. Для чистоты эксперимента можно удалить файлы которые имеются в данном каталоге или перенести их в другой каталог.

In [ ]:
from datenollm.jupiter_nb import get_full_path

get_full_path('')

'/content/drive/MyDrive/colab_data/dateno/'

### Запрос к LLM-агенту Query Assistant

Импортируем то что понадобится:

In [ ]:
import json
from datenollm.client import DatenoClient
from datenollm.jupiter_nb import ask_llm, QueryAssistantChatWidget, QueriesSelector

Стартуем клиент для доступа к HF Space, где работает нужный нам LLM-агент. М.б. придется позапускать несколько раз если будут ошибки -- HF Space засыпает если долго не использовался и запускается опять при обращении к нему. При таком раскладе -- заводиться может пинка с третьего.

In [ ]:
step1_agent = DatenoClient('datenoio/explainable-query-assistant', hf_token=HF_TOKEN)

Loaded as API: https://datenoio-explainable-query-assistant.hf.space ✔


Используем специализированный набор запросов Chain of Tought (CoT) для формирования методики и выбора наборов данных. Если глубокий анализ не нужен, переходим сразу к [запросу](#scrollTo=Y5xsUVNKIusc&line=1&uniqifier=1).

In [ ]:
# @title Задайте вопрос:
query = "the trade turnover between Armenia and Cyprus" # @param {"type":"string","placeholder":"Your query"}
history_file = "history.json"  # @param {"type":"string","placeholder":"File for saving history"}

if not query:
  raise ValueError('Query is empty')

main_query = query

In [ ]:
# @title Запустить CoT

# Отправить 1-ый CoT запрос к LLM:

if not main_query:
  raise ValueError('Query is empty')

queries = []

llm_query = f"""
I want to analyze the query:
{main_query}

As a professional economic analyst, tell me what methodological approaches are optimal for this?
"""

print('CoT step 1')
query, llm_response, history, error = ask_llm(step1_agent, llm_query, history_file=history_file)
queries.append(query)
rating_widget = QueryAssistantChatWidget(step1_agent, history_file)
display(rating_widget.display())

# Отправить 2-ой CoT запрос к LLM

if not main_query:
  raise ValueError('Query is empty')

llm_query = f"""
Divide these methods into those that require a high level of expertise and special knowledge, and those that are based directly on data analysis.
"""

print('CoT step 2')
query, llm_response, history, error = ask_llm(step1_agent, llm_query, history_file=history_file)
queries.append(query)
rating_widget = QueryAssistantChatWidget(step1_agent, history_file)
display(rating_widget.display())

# Отправить 3-ий CoT запрос к LLM

if not main_query:
  raise ValueError('Query is empty')

llm_query = f"""
For Data Analysis methods, list the required indicators and possible names of the corresponding data sets.
"""

print('CoT step 3')
query, llm_response, history, error = ask_llm(step1_agent, llm_query, history_file=history_file)
queries.append(query)
rating_widget = QueryAssistantChatWidget(step1_agent, history_file)
display(rating_widget.display())

# 4-ый CoT запрос к LLM -- проверка:

if not main_query:
  raise ValueError('Query is empty')

llm_query = f"""
Review the proposed indicators and data sets for the original query:
{main_query}

Provide a revised list of queries.
"""

print('CoT step 4')
query, llm_response, history, error = ask_llm(step1_agent, llm_query, history_file=history_file)
queries.append(query)
rating_widget = QueryAssistantChatWidget(step1_agent, history_file)
display(rating_widget.display())

CoT step 1
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'


CoT step 2
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'


CoT step 3
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'


CoT step 4
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'


In [ ]:
# @title Ну и дальше общаемся с LLM как обычно:
query = "for last 2 years" # @param {"type":"string","placeholder":"Your query"}

if not query:
  raise ValueError('Query is empty')

query, llm_response, history, error = ask_llm(step1_agent, query, history_file=history_file)
queries.append(query)
rating_widget = QueryAssistantChatWidget(step1_agent, history_file)
display(rating_widget.display())

history_file='/content/drive/MyDrive/colab_data/dateno/history.json'


### Выбор запросов для следующего этапа workflow

Тут можем выбрать набор запросов который будет передан на следующий этап workflow. Список для выбора будет формироваться из того набора запросов который вы сформировали последним -- CoT или обычный запрос к LLM.

In [ ]:
# @title Выберите запросы:
llm_response = json.loads(llm_response)
step1_selector = QueriesSelector(llm_response['queries'])
display(step1_selector)

QueriesSelector(children=(Button(description='Select All', style=ButtonStyle()), Button(description='Select No…

Сохраняем выбранное в файле для передачи на следующий этап workflow:

In [ ]:
# @title Сохраняем запросы для этапа поиска
file = 'workflow_step1.json' # @param {"type":"string","placeholder":"File to save the state of step 1"}

selected = step1_selector.get_selected_queries()
step1 = {'queries': queries, 'selected': selected}

with open(get_full_path(file), "w") as f:
  json.dump(step1, f)

## Второй этап workflow -- Dateno Search LLM-агент

LLM-агент Dateno Search преобразует запросы произвольной формы в поисковые запросы к базе датасетов через Dateno API.

Импортируем то что понадобится:

In [ ]:
import json

from datenollm.client import DatenoClient
from datenollm.jupiter_nb import (
    ChatWidget, DatenoSearchChatWidget, DatenoSearchQuerySelector,
    ask_llm, display_table, dateno2df, create_dateno_search_selector,
    ask_llm_and_create_selector, get_full_path,
    QueriesSelector, QuerySelector,
    )

Стартуем клиент для доступа к HF Space. **М.б. придется позапускать несколько раз если будут ошибки** -- HF Space засыпает если долго не использовался и запускается опять при обращении к нему. При таком раскладе -- заводится пинка с третьего.

In [ ]:
step2_agent = DatenoClient('datenoio/dateno-search', hf_token=HF_TOKEN)

Loaded as API: https://datenoio-dateno-search.hf.space ✔


### Передача запросов составленных LLM-агентом Query Assistant в Dateno Search

In [ ]:
# @title Считываем запросы сгенерированные Query Assitant'ом на предыдущем шаге:
file = 'workflow_step1.json' # @param {"type":"string","placeholder":"File with saved state of step 1"}

with open(get_full_path(file), 'r') as f:
    step1 = json.load(f)

step1_queries = step1['queries']
queries = step1_queries

In [ ]:
# @title Определяем функцию-обработчик.
def dateno_search(selected_query, context_file=None, history_file=None):
  global queries, step2_selector
  queries = step1_queries
  query = selected_query[0]['query']
  print(f'Query for Dateno search LLM-agent: "{query}"')
  query_text, result_json, history, error = ask_llm(step2_agent, query)

  if error:
    raise ValueError(f"Error: {error}")

  queries.append(query_text)
  response = json.loads(result_json)
  query_text, result_json, history, error = ask_llm(step2_agent, query, context_file=context_file, history_file=history_file)

  if error:
    raise ValueError(f"Error: {error}")

  print(f'Query for Dateno search LLM-agent: "{query_text}"')
  queries.append(query_text)
  step2_selector = DatenoSearchQuerySelector(step2_agent, response['queries'])

  step2_selector.display()

Далее выбираем запрос из набора сформированного на первом этапе и формируем поисковый запрос к Dateno. Если надо выбрать другой запрос или повторить прежний -- надо будет перезапустить эту ячейку заново.

In [ ]:
# @title Выбираем запрос и отправляем его в Dateno Search:
queries = []

step1_selector = QuerySelector(queries_data=step1['selected'], execute_func=dateno_search)
step1_selector.display()

Query for Dateno search LLM-agent: "IMF Direction of Trade Statistics Armenia Cyprus from 2023-09-06 to 2025-09-06"
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'
Query for Dateno search LLM-agent: "IMF Direction of Trade Statistics Armenia Cyprus from 2023-09-06 to 2025-09-06"


🔍 Executing search: "IMF Direction of Trade Statistics from 2023-09-06 to 2025-09-06" with filters: source.countries.name=Armenia

📊 Search Results for query: "IMF Direction of Trade Statistics from 2023-09-06 to 2025-09-06" with filters: source.countries.name=Armenia
   Filters: source.countries.name=Armenia
   Records found: 500


------------------------------


In [ ]:
# @title Сохраняем запросы и результаты поиска для этапа валидации
file = 'workflow_step2.json' # @param {"type":"string","placeholder":"File to save the state of step 2"}

step2_queries = []

# Queries formed in step 1 by the Query Assistant LLM-agent
for query in queries:
  step2_queries.append({'query': query})

# Chosen query from the final queries of step 1
selected = step1_selector.get_selected_queries()
step2_queries.append(selected[0])

# Query for the Dateno Search LLM-agent
selected = step2_selector.get_selected_queries()
step2_queries.append(selected[0])

step2_results = step2_selector.get_query_results()
step2_results = step2_results[0]['results']['hits']['hits']
step2 = [{'queries': step2_queries, 'results': step2_results},]

with open(get_full_path(file), "w") as f:
  json.dump(step2, f)

## Третий этап -- валидация результатов

In [ ]:
import json
from datenollm.client import DatenoFilter
from datenollm.jupiter_nb import get_full_path

In [ ]:
step3_agent = DatenoFilter('datenoio/dataset-validator', hf_token=HF_TOKEN)

Loaded as API: https://datenoio-dataset-validator.hf.space ✔


In [ ]:
# @title Считываем результаты 2-го этапа -- Dateno Search LLM-агента
file = 'workflow_step2.json' # @param {"type":"string","placeholder":"File with saved state of step 2"}

with open(get_full_path(file), 'r') as f:
    step2 = json.load(f)

In [ ]:
# @title Отправляем результаты 2-го этапа LLM-агенту валидатору
max_requests = 15 # @param {"type":"number", "placeholder":"Full number of records"}
max_requests_per_call=5 # @param {"type":"number", "placeholder":"Number of records per agent call"}
#max_tokens=2048 # @param {"type":"number", "placeholder":"Number of output tokens"}
max_tokens=2048

print('Sending data to LLM-agent for validation...')
result = step3_agent.filter(messages=[], history=[], data=step2, max_requests=15, max_requests_per_call=5, max_tokens=2048,
                       prompt=None, model=None, temperature=None, top_p=None, openai_api_base=None)
print('Accepted: {len(result["results"]["accepted"])}')
print('Declined: {len(result["results"]["declined"])}')

# Combine search data and validation results
data = step3_agent.filter2data(data=step2, combined_output=result)

# Show results
from IPython.display import display, HTML

html = step3_agent.results2html(data=data, verbose=True)
display(HTML(html))

Sending data to LLM-agent for validation...
Accepted: {len(result["results"]["accepted"])}
Declined: {len(result["results"]["declined"])}
